In [17]:
import pandas as pd
from plotly import graph_objs as go
import numpy as np

from tavis_cummings import Cavity, plot_populations

In [52]:
df = pd.read_csv("compiled-QMarina-data/QMarina-AQT-4Q.csv", index_col=0)
df = df.set_index(["Time", "Protocol", "Randomizations", "Identities"])
df.head()

,,,,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
Time,Protocol,Randomizations,Identities,,,,,,,,,,,,,,,,
0.00,RAW,1,0,0.050984,0.000456,0.013146,0.000406,0.010097,0.000013,0.003023,0.000135,0.785205,0.004714,0.023915,0.007711,0.075245,0.000834,0.023399,0.000717
0.06,RAW,1,0,0.090982,0.008129,0.092156,0.004433,0.015698,0.001441,0.011372,0.001295,0.563003,0.018957,0.035991,0.019312,0.098744,0.005485,0.027416,0.005586
0.12,RAW,1,0,0.110445,0.018383,0.137640,0.009343,0.035105,0.005053,0.015672,0.002615,0.500158,0.020734,0.071762,0.015112,0.033971,0.003547,0.017902,0.002558
0.18,RAW,1,0,0.075531,0.044555,0.200329,0.009750,0.099588,0.008033,0.019068,0.005962,0.363551,0.028193,0.049856,0.012690,0.046582,0.008596,0.024238,0.003478
0.24,RAW,1,0,0.109853,0.079092,0.188102,0.011857,0.110959,0.016458,0.029896,0.004643,0.302982,0.027324,0.052432,0.011789,0.033481,0.007138,0.012197,0.001797


In [70]:
cavity = Cavity(
    cavity_freq=1,
    num_emitters=3,
    num_photons=4,
    emitter_freq=1,
    gamma=0,
    kappa=3,
    g=4,
)

times = np.linspace(0, 3, 51, endpoint=True)
result = cavity.mesolve(
    cavity.emitter_state(0), times
)

sim_pops = result.expect[1:] + [1 - sum(result.expect[1:])]

labels = ["Em1", "Em2", "Em3", "Cav + env"]
bitstrs = ["1000", "0100", "0001", "0010"]

freqs = np.fft.rfftfreq(len(times), times[1] - times[0])

fig = go.Figure(layout=dict(title="FFT of QMARINA data", xaxis_title="Frequency", yaxis_title="Amplitude", margin=dict(t=40, l=20, r=20, b=20)))

raw_data = df.xs("RAW", level="Protocol")
rc_data = df.xs("RC", level="Protocol").xs(80, level="Randomizations")
nox_data = df.xs("NOX", level="Protocol").xs(6, level="Identities")
times = raw_data.index.get_level_values("Time").unique()

for label, bitstr, sim_pop in zip(labels, bitstrs, sim_pops):
    fft = np.abs(np.fft.rfft(sim_pop))
    fig.add_trace(go.Scatter(x=freqs, y=fft, name=label + " (qutip)"))

    raw_fft = np.abs(np.fft.rfft(raw_data[bitstr]))
    fig.add_trace(go.Scatter(x=freqs, y=raw_fft, name=label + " (AQT raw)"))

    rc_fft = np.abs(np.fft.rfft(rc_data[bitstr]))
    fig.add_trace(go.Scatter(x=freqs, y=rc_fft, name=label + " (AQT RC)"))

    nox_fft = np.abs(np.fft.rfft(nox_data[bitstr]))
    fig.add_trace(go.Scatter(x=freqs, y=nox_fft, name=label + " (AQT RC + NOX)"))

fig.show()

In [67]:
fig = go.Figure(layout=dict(title="QMARINA populations", xaxis_title="Time", yaxis_title="Population", margin=dict(t=40, l=20, r=20, b=20)))

for label, bitstr, sim_pop in zip(labels, bitstrs, sim_pops):
    fig.add_trace(go.Scatter(x=times, y=sim_pop, name=label + " (qutip)"))
    fig.add_trace(go.Scatter(x=times, y=raw_data[bitstr], name=label + " (AQT raw)"))
    fig.add_trace(go.Scatter(x=times, y=rc_data[bitstr], name=label + " (AQT RC)"))
    fig.add_trace(go.Scatter(x=times, y=nox_data[bitstr], name=label + " (AQT RC + NOX)"))

fig.show()

In [8]:
df.index.get_level_values("Protocol").unique()

Index(['RAW', 'RC', 'NOX'], dtype='object', name='Protocol')

In [20]:
raw_data = df.xs("RAW", level="Protocol")
times = raw_data.index.get_level_values("Time")
fig = go.Figure(layout=dict(title="No error mitigation", yaxis_title="Population", xaxis_title="Time", margin=dict(t=50, l=20, r=20, b=20)))
for col in raw_data.columns:
    fig.add_trace(go.Scatter(x=times, y=raw_data[col], name=col))
fig.show()

In [34]:
rc_data = df.xs("RC", level="Protocol")
rc_data.index.get_level_values("Randomizations").unique()

Index([40, 80], dtype='int64', name='Randomizations')

In [36]:
rc_data = df.xs("RC", level="Protocol").xs(80, level="Randomizations")
times = rc_data.index.get_level_values("Time")
fig = go.Figure(layout=dict(title="Randomized compiling", yaxis_title="Population", xaxis_title="Time", margin=dict(t=50, l=20, r=20, b=20)))
for col in rc_data.columns:
    fig.add_trace(go.Scatter(x=times, y=rc_data[col], name=col))
fig.show()

In [41]:
nox_data = df.xs("NOX", level="Protocol")
nox_data.index.get_level_values("Identities").unique()

Index([1, 2, 3, 4, 6], dtype='int64', name='Identities')

In [41]:
nox_data = df.xs("NOX", level="Protocol").xs(6, level="Identities")
times = rc_data.index.get_level_values("Time")
fig = go.Figure(layout=dict(title="NOX + RC", yaxis_title="Population", xaxis_title="Time", margin=dict(t=50, l=20, r=20, b=20)))
for col in nox_data.columns:
    fig.add_trace(go.Scatter(x=times, y=nox_data[col], name=col))
fig.show()

In [32]:
import plotly.graph_objects as go

# Initialize figure
fig = go.Figure(layout=dict(title="NOX + RC", 
                            yaxis_title="Population", 
                            xaxis_title="Time", 
                            margin=dict(t=50, l=20, r=20, b=20)))

# Identities values
identities = [1, 2, 3, 4, 6]

# Add traces for each identity
for identity in identities:
    nox_data = df.xs("NOX", level="Protocol").xs(identity, level="Identities")
    times = nox_data.index.get_level_values("Time")
    
    for col in nox_data.columns:
        fig.add_trace(go.Scatter(x=times, y=nox_data[col], name=f"{col} (Identity {identity})",
                                 visible=(identity == identities[0])))  # Only the first identity is visible initially

# Create and add slider
steps = []
for i, identity in enumerate(identities):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f"NOX + RC: Identity {identity}"}],
        label=f"Identity {identity}"
    )
    
    for j in range(i * len(nox_data.columns), (i + 1) * len(nox_data.columns)):
        step["args"][0]["visible"][j] = True  # Toggle i-th trace to "visible"

    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Identity: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders)

# Show the figure
fig.show()
